In [ ]:
import numpy as np
import pandas as pd

In [ ]:
credits = pd.read_csv("../input/tmdb-movie-metadata/tmdb_5000_credits.csv")
credits.shape

In [ ]:
movies = pd.read_csv("../input/tmdb-movie-metadata/tmdb_5000_movies.csv")
movies.shape

In [ ]:
credits.head()

In [ ]:
credits.columns

In [ ]:
movies.head()

In [ ]:
movies.columns

 In credit datasets we are getting column i.e movie id and in movies dataset we are getting the movie id named id only so we will rename the movie id as id and merge into movies dataset  

In [ ]:
rename_movie_id = credits.rename(index = str,columns = {"movie_id":"id"})
movies_merge = movies.merge(rename_movie_id,on= 'id')
movies_merge.head()

 Now,we merge the both column into movie dataset

In [ ]:
movies.isnull().sum()

In [ ]:
movies.info()

# Now we have to clean the data

So here we are getting in homepage column 3091 null values and in tagline column we are getting 844 null values we will drop those columns which is not needed

In [ ]:
clean_movies_data = movies_merge.drop(columns = ['homepage','title_x','title_y','status','production_countries','production_companies','spoken_languages','crew','cast','runtime','revenue','popularity','budget'])
clean_movies_data.head()

In [ ]:
clean_movies_data.head()['overview']

In [ ]:
clean_movies_data.head()['original_title']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df = 3, max_features = None,
                      strip_accents="unicode",analyzer = "word",token_pattern = r'\w{1,}',
                      ngram_range=(1, 3),
                      stop_words = "english")

clean_movies_data['overview'] = clean_movies_data['overview'].fillna('')
#Filling NaNs with empty string                      


In [ ]:
#Now fitting the TF-IDF on the 'overview' text
tfv_matrix = tfv.fit_transform(clean_movies_data['overview'])

In [ ]:
tfv_matrix

In [ ]:
tfv_matrix.shape

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel

sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [ ]:
sig[0]

In [ ]:
indices = pd.Series(clean_movies_data.index, index = clean_movies_data['original_title']).drop_duplicates()

In [ ]:
indices

In [ ]:
indices['Newlyweds']

In [ ]:
list(enumerate(sig[indices['Newlyweds']]))

In [ ]:
sorted(list(enumerate(sig[indices['Newlyweds']])),key=lambda x:x[1],reverse = True)

In [ ]:
def give_rec(title,sig=sig):
    #we get the index according to corresponding_title
    idx = indices[title]
    
    #now, will get the pairwise similarity scores
    sig_scores = list(enumerate(sig[idx]))
    
    #sort the movies
    sig_scores = sorted(sig_scores,key=lambda x:x[1],reverse=True)
    
    #scores of the 20 most similar movies
    sig_scores = sig_scores[1:21]
    
    #Movie indices
    movie_indices = [i[0] for i in sig_scores]
    
    #top 20 most similar movies
    return clean_movies_data['original_title'].iloc[movie_indices]

In [ ]:
give_rec('Shanghai Calling')